In [3]:
import pandas as pd
import numpy as np
import simpy
import random

menu_df = pd.read_csv('menu.csv')
customer_base_df=pd.read_csv("customer_base.csv")
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

try:
    menu_df['Gross Profit Per Item']=pd.to_numeric(menu_df['Gross Profit Per Item'].str.replace('$',''))
except:
    pass
customer_base_df.to_csv('customer_base.csv', index=False)


np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

DAYS_IN_SIMULATION = 30 * 12
SIMULATION_TIME_PER_DAY = 8 * 60  # 8 hours per day in minutes
ARRIVAL_RATE = 1  # Average time between arrivals
MAX_QUEUE_LENGTH_REGULAR = 5  # Maximum queue length before regular customers start balking
MAX_QUEUE_LENGTH_IMPATIENT = 3  # Maximum queue length for impatient customers
RENEGING_TIME_REGULAR = 8  # Time after which a regular customer may renege
RENEGING_TIME_IMPATIENT = 3  # Time for impatient customers
IMPATIENCE_PROBABILITY = 0.15  # Probability that a customer is impatient on a given visit

customer_records = []

def choose_items(menu, num_items):
    chosen_items = random.choices(menu['ID'], weights=menu['Popularity'], k=num_items)
    return chosen_items

class Customer:
    def __init__(self, env, customer_id, queue, queue_name, day, is_subscriber, is_impatient):
        self.env = env
        self.customer_id = customer_id
        self.queue = queue
        self.queue_name = queue_name
        self.day = day
        self.is_subscriber = is_subscriber
        self.is_impatient = is_impatient

    def order(self):
        num_items_ordered = max(1, min(8, int(np.random.normal(3, 1))))
        ordered_items = choose_items(menu_df, num_items_ordered)
        total_prep_time = sum(menu_df[menu_df['ID'].isin(ordered_items)]['Prep_Time'])
        max_queue_length = MAX_QUEUE_LENGTH_IMPATIENT if self.is_impatient else MAX_QUEUE_LENGTH_REGULAR
        reneging_time = RENEGING_TIME_IMPATIENT if self.is_impatient else RENEGING_TIME_REGULAR

        record = {
            'customer_id': self.customer_id,
            'is_subscriber': self.is_subscriber,
            'is_impatient': self.is_impatient,
            'queue_name': self.queue_name,
            'day': self.day,
            'arrival_time': self.env.now,
            'num_items_ordered': num_items_ordered
        }

        for i in range(1, 8):
            record[f'item{i}'] = ordered_items[i - 1] if i <= len(ordered_items) else None

        with self.queue.request() as request:
            if len(self.queue.queue) > max_queue_length:
                record['action'] = 'balked'
                customer_records.append(record)
                return

            yield request | self.env.timeout(reneging_time)
            yield self.env.timeout(total_prep_time)

            record['departure_time'] = self.env.now
            record['action'] = 'served'
            customer_records.append(record)

def burger_shop(env, regular_queue, premium_queue, day, customer_base, premium_price):
    while True:
        arrival_interval = random.expovariate(1.0 / ARRIVAL_RATE)
        yield env.timeout(arrival_interval)

        selected_customer = customer_base.sample(1).iloc[0]
        customer_id = selected_customer['CustomerID']
        price_tolerance = selected_customer['PriceTolerance']
        is_subscriber = selected_customer['IsSubscriber'] or (price_tolerance >= premium_price)

        is_impatient = random.random() < IMPATIENCE_PROBABILITY
        queue = premium_queue if is_subscriber else regular_queue
        queue_name = 'premium_queue' if is_subscriber else 'regular_queue'

        customer = Customer(env, customer_id, queue, queue_name, day, is_subscriber, is_impatient)
        env.process(customer.order())



def calculate_profit(row, menu):
    total_profit = 0.0
    for i in range(1, 8): 
        item_id = row[f'item{i}']
        if item_id!=0:
            profit = menu.loc[menu['ID'] == item_id, 'Gross Profit Per Item'].values[0]
            total_profit += profit
    return total_profit



def burger_shop(env, regular_queue, premium_queue, day, customer_base, premium_price):
    while True:
        arrival_interval = random.expovariate(1.0 / ARRIVAL_RATE)
        yield env.timeout(arrival_interval)

        selected_customer = customer_base.sample(1).iloc[0]
        customer_id = selected_customer['CustomerID']
        is_impatient = random.random() < IMPATIENCE_PROBABILITY
        price_tolerance = selected_customer['PriceTolerance']
        is_subscriber = selected_customer['IsSubscriber'] or (price_tolerance >= premium_price)

        queue = premium_queue if is_subscriber else regular_queue
        queue_name = 'premium_queue' if is_subscriber else 'regular_queue'

        customer = Customer(env, customer_id, queue, queue_name, day, is_subscriber, is_impatient)
        env.process(customer.order())

def run_simulation_for_price(premium_price, customer_base_df):
    global customer_records
    customer_records = []

    for day in range(1, DAYS_IN_SIMULATION + 1):
        env = simpy.Environment()
        regular_queue = simpy.Resource(env, capacity=1)
        premium_queue = simpy.Resource(env, capacity=1)
        env.process(burger_shop(env, regular_queue, premium_queue, day, customer_base_df, premium_price))
        env.run(until=SIMULATION_TIME_PER_DAY)

    simulation_df = pd.DataFrame(customer_records)
    for i in range(1, 8):
        simulation_df[f'item{i}'] = simulation_df[f'item{i}'].fillna(0).astype(int)
    simulation_df['profit'] = simulation_df.apply(lambda row: calculate_profit(row, menu_df), axis=1)
    profit_per_subscriber = premium_price
    sub_profit = simulation_df[simulation_df['is_subscriber']]['customer_id'].nunique() * profit_per_subscriber
    food_profit=simulation_df['profit'].sum()
    total_profit=sub_profit+food_profit
    return total_profit

def optimize_membership_price(customer_base_df):
    def objective_function(price):
        return -run_simulation_for_price(price[0], customer_base_df)

    bounds = [(10.00, 25.00)] 

    initial_guess = [18.00]

    result = minimize(objective_function, initial_guess, bounds=bounds)

    return result, result.x[0]

def run_simulation_for_price(premium_price, customer_base_df):
    global customer_records
    customer_records = []

    for day in range(1, DAYS_IN_SIMULATION + 1):
        env = simpy.Environment()
        regular_queue = simpy.Resource(env, capacity=1)
        premium_queue = simpy.Resource(env, capacity=1)
        env.process(burger_shop(env, regular_queue, premium_queue, day, customer_base_df, premium_price))
        env.run(until=SIMULATION_TIME_PER_DAY)

    simulation_df = pd.DataFrame(customer_records)
    for i in range(1, 8):
        simulation_df[f'item{i}'] = simulation_df[f'item{i}'].fillna(0).astype(int)
    simulation_df['profit'] = simulation_df.apply(lambda row: calculate_profit(row, menu_df), axis=1)
    profit_per_subscriber = premium_price
    sub_profit = simulation_df[simulation_df['is_subscriber']]['customer_id'].nunique() * profit_per_subscriber
    food_profit=simulation_df['profit'].sum()
    total_profit=sub_profit+food_profit
    simulation_df['wait_time'] = simulation_df['departure_time'] - simulation_df['arrival_time']
    return total_profit

def run_simulation_for_dataset(premium_price, customer_base_df):
    global customer_records
    customer_records = []

    for day in range(1, DAYS_IN_SIMULATION + 1):
        env = simpy.Environment()
        regular_queue = simpy.Resource(env, capacity=1)
        premium_queue = simpy.Resource(env, capacity=1)
        env.process(burger_shop(env, regular_queue, premium_queue, day, customer_base_df, premium_price))
        env.run(until=SIMULATION_TIME_PER_DAY)

    simulation_df = pd.DataFrame(customer_records)
    for i in range(1, 8):
        simulation_df[f'item{i}'] = simulation_df[f'item{i}'].fillna(0).astype(int)
    simulation_df['profit'] = simulation_df.apply(lambda row: calculate_profit(row, menu_df), axis=1)
    simulation_df['wait_time'] = simulation_df['departure_time'] - simulation_df['arrival_time']
    return simulation_df

test1=run_simulation_for_dataset(15,customer_base_df)
test1

,customer_id,is_subscriber,is_impatient,queue_name,day,arrival_time,num_items_ordered,item1,item2,item3,item4,item5,item6,item7,departure_time,action,profit,wait_time
0,3827,True,False,premium_queue,1,2.111422,1,1,0,0,0,0,0,0,2.611422,served,5.6,0.500000
1,1502,False,True,regular_queue,1,1.020060,4,2,7,7,8,0,0,0,3.520060,served,17.5,2.500000
2,483,False,True,regular_queue,1,1.341684,3,1,2,5,0,0,0,0,5.520060,served,15.3,4.178376
3,3903,True,False,premium_queue,1,3.769160,2,3,1,0,0,0,0,0,5.769160,served,9.6,2.000000
4,1106,False,True,regular_queue,1,1.889731,3,6,6,2,0,0,0,0,6.639731,served,14.2,4.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170933,4081,False,False,regular_queue,360,472.980960,2,1,2,0,0,0,0,0,474.480960,served,12.6,1.500000
170934,1083,True,False,premium_queue,360,473.692730,1,2,0,0,0,0,0,0,474.692730,served,7.0,1.000000
170935,2986,False,True,regular_queue,360,474.069705,2,4,2,0,0,0,0,0,477.480960,served,12.6,3.411254
170936,4630,True,False,premium_queue,360,475.130662,3,5,1,3,0,0,0,0,477.630662,served,12.3,2.500000


In [2]:
import pandas as pd
import numpy as np
import simpy
import random
from concurrent.futures import ProcessPoolExecutor, as_completed

# Constants
RANDOM_SEED = 42
DAYS_IN_SIMULATION = 30 * 12
SIMULATION_TIME_PER_DAY = 8 * 60
ARRIVAL_RATE = 1
MAX_QUEUE_LENGTH_REGULAR = 5
MAX_QUEUE_LENGTH_IMPATIENT = 3
RENEGING_TIME_REGULAR = 8
RENEGING_TIME_IMPATIENT = 3
IMPATIENCE_PROBABILITY = 0.15
NUM_PROCESSES = 96  # Adjust based on performance profiling

# Set random seed for reproducibility
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Load data
menu_df = pd.read_csv('menu.csv')
menu_df['Gross Profit Per Item'] = pd.to_numeric(menu_df['Gross Profit Per Item'].str.replace('$', ''), errors='coerce')
customer_base_df = pd.read_csv("customer_base.csv")

# Simulation functions
def choose_items(menu, num_items):
    return random.choices(menu['ID'], weights=menu['Popularity'], k=num_items)

class Customer:
    def __init__(self, env, customer_id, queue, queue_name, day, is_subscriber, is_impatient):
        self.env = env
        self.customer_id = customer_id
        self.queue = queue
        self.queue_name = queue_name
        self.day = day
        self.is_subscriber = is_subscriber
        self.is_impatient = is_impatient

    def order(self):
        num_items_ordered = max(1, min(8, int(np.random.normal(3, 1))))
        ordered_items = choose_items(menu_df, num_items_ordered)
        total_prep_time = sum(menu_df[menu_df['ID'].isin(ordered_items)]['Prep_Time'])
        max_queue_length = MAX_QUEUE_LENGTH_IMPATIENT if self.is_impatient else MAX_QUEUE_LENGTH_REGULAR
        reneging_time = RENEGING_TIME_IMPATIENT if self.is_impatient else RENEGING_TIME_REGULAR

        record = {
            'customer_id': self.customer_id,
            'is_subscriber': self.is_subscriber,
            'is_impatient': self.is_impatient,
            'queue_name': self.queue_name,
            'day': self.day,
            'arrival_time': self.env.now,
            'num_items_ordered': num_items_ordered
        }

        for i in range(1, 8):
            record[f'item{i}'] = ordered_items[i - 1] if i <= len(ordered_items) else None

        with self.queue.request() as request:
            if len(self.queue.queue) > max_queue_length:
                record['action'] = 'balked'
                return record

            yield request | self.env.timeout(reneging_time)
            yield self.env.timeout(total_prep_time)

            record['departure_time'] = self.env.now
            record['action'] = 'served'
            return record

def burger_shop(env, regular_queue, premium_queue, day, customer_base, premium_price):
    while True:
        arrival_interval = random.expovariate(1.0 / ARRIVAL_RATE)
        yield env.timeout(arrival_interval)

        selected_customer = customer_base.sample(1).iloc[0]
        customer_id = selected_customer['CustomerID']
        price_tolerance = selected_customer['PriceTolerance']
        is_subscriber = selected_customer['IsSubscriber'] or (price_tolerance >= premium_price)

        is_impatient = random.random() < IMPATIENCE_PROBABILITY
        queue = premium_queue if is_subscriber else regular_queue
        queue_name = 'premium_queue' if is_subscriber else 'regular_queue'

        customer = Customer(env, customer_id, queue, queue_name, day, is_subscriber, is_impatient)
        yield env.process(customer.order())

def calculate_profit(row, menu):
    total_profit = 0.0
    for i in range(1, 8): 
        item_id = row[f'item{i}']
        if item_id != 0:
            profit = menu.loc[menu['ID'] == item_id, 'Gross Profit Per Item'].values[0]
            total_profit += profit
    return total_profit

def run_simulation_for_price(premium_price, customer_base_df):
    customer_records = []

    for day in range(1, DAYS_IN_SIMULATION + 1):
        env = simpy.Environment()
        regular_queue = simpy.Resource(env, capacity=1)
        premium_queue = simpy.Resource(env, capacity=1)
        env.process(burger_shop(env, regular_queue, premium_queue, day, customer_base_df, premium_price))
        while not env.peek() == float('inf'):
            env.step()
            record = env.active_process.value  # Get the result from the process
            if record:  # If record is not None
                customer_records.append(record)

    simulation_df = pd.DataFrame(customer_records)
    for i in range(1, 8):
        simulation_df[f'item{i}'] = simulation_df[f'item{i}'].fillna(0).astype(int)
    simulation_df['profit'] = simulation_df.apply(lambda row: calculate_profit(row, menu_df), axis=1)
    profit_per_subscriber = premium_price
    sub_profit = simulation_df[simulation_df['is_subscriber']]['customer_id'].nunique() * profit_per_subscriber
    food_profit = simulation_df['profit'].sum()
    total_profit = sub_profit + food_profit
    return total_profit

# Parallel profit calculation
def simulation_wrapper(price):
    return run_simulation_for_price(price, customer_base_df)

def parallel_profit_calculation(start_price, end_price):
    prices = np.arange(start_price, end_price + 0.01, 0.01)
    future_to_price = {}
    
    with ProcessPoolExecutor(max_workers=NUM_PROCESSES) as executor:
        for price in prices:
            future = executor.submit(simulation_wrapper, price)
            future_to_price[future] = price

        results = []
        for future in as_completed(future_to_price):
            price = future_to_price[future]
            profit = future.result()
            results.append((price, profit))
            print(f"Completed simulation for price: ${price:.2f}")

    results.sort(key=lambda x: x[0])
    return pd.DataFrame(results, columns=['price', 'profit'])

# Running the simulation
sim_df1 = run_simulation_for_price(18, customer_base_df)
sim_df1.to_csv('simulation_results.csv', index=False)

AttributeError: 'NoneType' object has no attribute 'value'